# 강의 코드와 다르니 다시 한 번 확인해볼 것

# 논리게이트 - AND, OR, NAND, XOR

##  AND, OR, NAND, XOR 논리테이블(Logic Table)은 입력데이터(x1, x2), 정답데이터 t (0 또는 1)인 머신러닝 training data와 개념적으로 동일함

## 즉, 논리게이트는 손실함수로 cross-entropy를 이용해서 Logistic Regression(Classification) 알고리즘으로 데이터를 분류하고 결과를 예측할 수 있음

In [16]:
# external function (sigmoid, numerical_derivative)
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def numerical_derivative(f, x): #x는 모든 변수를 포함하고 있는 numpy객체(배열, 행렬...)
    delta_x = 1e-4 #0.0001
    grad = np.zeros_like(x)

    it = np.nditer(x, flags = ['multi_index'], op_flags=['readwrite']) #모든 입력변수에 대해 편미분하기 위해 iterator 사용
    
    while not it.finished: 
        idx = it.multi_index
        
        tmp_val = x[idx] 
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) #f(x+delta_x)
        
        x[idx] = tmp_val - delta_x
        fx2 = f(x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val
        it.iternext()
    
    return grad

In [32]:
#logicgate class

class LogicGate:
    
    def __init__(self, gate_name, xdata, tdata): #xdata, tdata => numpy.array(...)
        
        self.name = gate_name
        
        #입력 데이터, 정답 데이터 초기화
        self.__xdata = xdata.reshape(4, 2) # 입력데이터는 (0, 0), (0, 1), (1, 0), (1, 1) 총 4가지
        self.__tdata = tdata.reshape(4, 1)
        
        #가중치 W, 바이어스 b 초기화
        self.__W = np.random.rand(2, 1) #weight, 2x1 matrix
        self.__b = np.random.rand(1)
        
        #가중치w와 바이어스b는 모두 priavate으로 선언되어 있다는 것
        #왜냐하면 파이선 클래스의 variable은 기본적으로  public이기 때문에 외부에서 바꿀 가능성이 있음
        #하지만 w, b는 외부에서 바꾸면 안되기 때문에 내부 method만을 통해서 바꿀 수 있게 안전장치 설치
        
        #학습률 learning rate 초기화
        self.__learning_rate = 1e-2 #발산하는 경우 더 작게 (1e-3, 1e-4...)

    #손실함수
    def __loss_func(self):

        delta = 1e-7 # log 무한대 발산 방지 / y가 0이나 1인 경우 무한대가 되어 버리기 때문에 이를 방지

        z = np.dot(self.__xdata, self.__W) + self.__b
        y = sigmoid(z)

        #cross-entropy

        return -np.sum(self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y) + delta))
    
    #손실 값 계산
    
    def error_val(self):
        delta = 1e-7 #log 무한대 발산 방지

        z = np.dot(self.__xdata, self.__W) + self.__b
        y = sigmoid(z)

        #cross-entropy
        return -np.sum(self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y) + delta))
    #수치미분을 이용하여 손실함수가 최소가 될 때까지 학습하는 함수

    def train(self):

        f = lambda x : self.__loss_func()

        print("initial error value =", self.error_val())

        for step in range(8001):
            self.__W -= self.__learning_rate + numerical_derivative(f, self.__W)
            self.__b -= self.__learning_rate + numerical_derivative(f, self.__b)

            if (step % 400 == 0):
                print("step = ", step, "error value = ", self.error_val())

    # 미래 값 예측 함수

    def predict(self, input_data):
        z = np.dot(input_data, self.__W) + self.__b
        y = sigmoid(z)

        if y > 0.5:
            result = 1 # True
        else:
            result = 0 # False

        return y, result

In [36]:
#입력 데이터 / 정답 데이터

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1]) #AND 정답데이터

AND_obj = LogicGate("AND_GATE", xdata, tdata)

AND_obj.train()

initial error value = 3.6364490485663126
step =  0 error value =  2.3160981658765643
step =  400 error value =  0.04645309202166194
step =  800 error value =  0.027473141903401874
step =  1200 error value =  0.021212742186327783
step =  1600 error value =  0.01816038384385633
step =  2000 error value =  0.016375719182666382
step =  2400 error value =  0.015213865840320268
step =  2800 error value =  0.014401346480405931
step =  3200 error value =  0.013803251851151607
step =  3600 error value =  0.013345702506689303
step =  4000 error value =  0.012985002459024632
step =  4400 error value =  0.012693728420059384
step =  4800 error value =  0.01245383800762601
step =  5200 error value =  0.012252994949942529
step =  5600 error value =  0.012082490090875572
step =  6000 error value =  0.011936006348985777
step =  6400 error value =  0.011808854315935879
step =  6800 error value =  0.011697482310930263
step =  7200 error value =  0.011599152727559752
step =  7600 error value =  0.01151172

In [34]:
# AND Gate prediction
print(AND_obj.name, "\n")

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for input_data in test_data:
    (sigmoid_val, logical_val) = AND_obj.predict(input_data) 
    print(input_data, " = ", logical_val, "\n")

AND_GATE 

[0 0]  =  0 

[0 1]  =  0 

[1 0]  =  0 

[1 1]  =  1 



In [37]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

OR_obj = LogicGate("OR_GATE", xdata, tdata)

OR_obj.train()

initial error value = 1.9168827921827731
step =  0 error value =  1.4460862724242567
step =  400 error value =  0.041941213816685606
step =  800 error value =  0.03309941765414632
step =  1200 error value =  0.03107352376720691
step =  1600 error value =  0.03045688073026242
step =  2000 error value =  0.030253360185853045
step =  2400 error value =  0.03018439955827074
step =  2800 error value =  0.030160824970061128
step =  3200 error value =  0.030152741441291375
step =  3600 error value =  0.030149966791482335
step =  4000 error value =  0.030149014061534987
step =  4400 error value =  0.030148686882342974
step =  4800 error value =  0.03014857452158534
step =  5200 error value =  0.030148535933341828
step =  5600 error value =  0.030148522680856764
step =  6000 error value =  0.030148518129443913
step =  6400 error value =  0.03014851656641042
step =  6800 error value =  0.03014851602951671
step =  7200 error value =  0.030148515845269203
step =  7600 error value =  0.030148515781

In [38]:
# OR Gate prediction
print(OR_obj.name, "\n")

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for input_data in test_data:
    (sigmoid_val, logical_val) = OR_obj.predict(input_data) 
    print(input_data, " = ", logical_val, "\n")

OR_GATE 

[0 0]  =  0 

[0 1]  =  1 

[1 0]  =  1 

[1 1]  =  1 



In [39]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

NAND_obj = LogicGate("NAND_GATE", xdata, tdata)

NAND_obj.train()

initial error value = 2.7621611123019223
step =  0 error value =  2.077007963148196
step =  400 error value =  0.036879191135763864
step =  800 error value =  0.019373291491657268
step =  1200 error value =  0.015025561408556324
step =  1600 error value =  0.013801501880631787
step =  2000 error value =  0.01346795902970297
step =  2400 error value =  0.013379403190639232
step =  2800 error value =  0.013356111524461915
step =  3200 error value =  0.01335000202582214
step =  3600 error value =  0.013348400649521202
step =  4000 error value =  0.013347980989402349
step =  4400 error value =  0.013347871017996484
step =  4800 error value =  0.013347842200663783
step =  5200 error value =  0.01334783464925277
step =  5600 error value =  0.01334783267023341
step =  6000 error value =  0.013347832151487849
step =  6400 error value =  0.013347832015648113
step =  6800 error value =  0.013347831980087689
step =  7200 error value =  0.013347831970664806
step =  7600 error value =  0.0133478319

In [40]:
# NAND Gate prediction
print(NAND_obj.name, "\n")

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for input_data in test_data:
    (sigmoid_val, logical_val) = NAND_obj.predict(input_data) 
    print(input_data, " = ", logical_val, "\n")

NAND_GATE 

[0 0]  =  1 

[0 1]  =  1 

[1 0]  =  1 

[1 1]  =  0 



In [41]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0]) #XOR 정답 데이터


XOR_obj = LogicGate("XOR_GATE", xdata, tdata)

# XOR Gate 를 보면, 손실함수 값이 2.7 근처에서 더 이상 감소하지 않는것을 볼수 있음
XOR_obj.train()

initial error value = 3.529670423947959
step =  0 error value =  2.7793188961226716
step =  400 error value =  2.7727379354256856
step =  800 error value =  2.7727379354256994
step =  1200 error value =  2.772737935425646
step =  1600 error value =  2.772737935425614
step =  2000 error value =  2.7727379354256056
step =  2400 error value =  2.772737935425619
step =  2800 error value =  2.772737935425609
step =  3200 error value =  2.7727379354256447
step =  3600 error value =  2.772737935425592
step =  4000 error value =  2.772737935425627
step =  4400 error value =  2.772737935425596
step =  4800 error value =  2.772737935425698
step =  5200 error value =  2.772737935425556
step =  5600 error value =  2.772737935425613
step =  6000 error value =  2.7727379354255604
step =  6400 error value =  2.772737935425618
step =  6800 error value =  2.7727379354256088
step =  7200 error value =  2.7727379354256443
step =  7600 error value =  2.7727379354256354
step =  8000 error value =  2.772737

In [44]:
# XOR Gate prediction => 예측이 되지 않음
print(XOR_obj.name, "\n")

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for input_data in test_data:
    (sigmoid_val, logical_val) = XOR_obj.predict(input_data) 
    print(input_data, " = ", logical_val, "\n")

XOR_GATE 

[0 0]  =  1 

[0 1]  =  0 

[1 0]  =  0 

[1 1]  =  0 



In [ ]:

# XOR 을 NAND + OR => AND 조합으로 계산함
input_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

s1 = []    # NAND 출력
s2 = []    # OR 출력

new_input_data = []  # AND 입력
final_output = []    # AND 출력

for index in range(len(input_data)):
    
    s1 = NAND_obj.predict(input_data[index])  # NAND 출력
    s2 = OR_obj.predict(input_data[index])    # OR 출력
    
    new_input_data.append(s1[-1])    # AND 입력
    new_input_data.append(s2[-1])    # AND 입력
    
    (sigmoid_val, logical_val) = AND_obj.predict(np.array(new_input_data))
    
    final_output.append(logical_val)    # AND 출력, 즉 XOR 출력    
    new_input_data = []    # AND 입력 초기화


for index in range(len(input_data)):    
    print(input_data[index], " = ", final_output[index], end='')
    print("\n")